## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. Please submit only the `*.ipynb` file.

4. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

5. Make sure to use Python 3, not Python 2.

Fill your group name and collaborators below:

In [2]:
# GROUPNAME = "42"
COLLABORATORS = "Viet Duc Nguyen, Laura Schöne, Jacky Behrendt"
#               (っ◕‿◕)っ           ( ^-^)_旦      ヽ(·̿̿Ĺ̯̿̿·̿✿)ﾉ

---

# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree. Should return `"less"` or `"more"`.
* Test your function on the tuple `('yes', 31, 'good')`,

In [3]:
def decision(x):
    # >>>>> YOUR CODE HERE
    health = None
    if x[0] == 'yes':
        if int(x[1]) < 29.5:
            health = 'less'
        else:
            health = 'more'
    elif x[0] != 'yes':
        if x[2] == 'good':
            health = 'less'
        else:
            health = 'more'
    return health 
    # <<<<< END YOUR CODE

In [4]:
# Test
x = ('yes', 31, 'good')
assert decision(x) == 'more'


## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples. Make sure that the tuples have the same format as above, e.g. `('yes', 31, 'good')`.
* Make extra note of the datatype of each element

In [5]:
def gettest():
    # >>>>> YOUR CODE HERE
    with open('health-test.txt', 'r') as f:
        D = []
        for line in f:
            A = line.split(',')
            D.append((A[0], int(A[1]), A[2][:-1]))
        return D
    # <<<<< END YOUR CODE

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and return the ratio of them that are classified as "more".

In [73]:
def evaluate_testset():
    # >>>>> YOUR CODE HERE
    H = gettest()
    moreSet = list(filter(lambda x: decision(x) == 'more', H))
    return len(moreSet)/len(H)
    # <<<<< END YOUR CODE

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [32]:
def gettrain():
    # >>>>> YOUR CODE HERE
    with open('health-train.txt', 'r') as f:
        D = []
        for line in f:
            A = line.split(',')
            D.append(((A[0], int(A[1]), A[2]), A[3][:-1])) #((smoke, age, diet), label)
    return D
    # <<<<< END YOUR CODE

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a, b) = (a[0] != b[0]) + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] != b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes', 31, 'good')`

In [76]:
def neighbor(a, trainset):
    # >>>>> YOUR CODE HERE
        minDist = float('Inf')
        health = None 
        for c in trainset: # (feature vectore, label)
            feature_vector = c[0]
            label = c[1]
            dist = (int(a[0] != feature_vector[0])) + ((a[1] - feature_vector[1]) / 50.0) ** 2 + (int(a[2] != feature_vector[2]))
            if dist < minDist:
                health = label
                minDist = dist

        return health 
    # <<<<< END YOUR CODE

In [77]:
# Test
x = ('yes', 31, 'good')
assert neighbor(x, gettrain()) == "more"


* Apply both the decision tree and nearest neighbor classifiers on the test set, and return the list of data point(s) for which the two classifiers disagree, and with which probability it happens.
* A data point should look like above, e.g. `('yes', 31, 'good')`.

In [79]:
def compare():
    # >>>>> YOUR CODE HERE
    testData = gettest()
    trainData = gettrain()
    
    res = list(map(lambda x: (x, decision(x), neighbor(x, trainData)), testData))
    wrongLabels = list(filter(lambda x: x[1] != x[2], res))
            
    probability = len(wrongLabels) / len(testData)
    Xdisagree = list(map(lambda x: x[0], wrongLabels))
    # <<<<< END YOUR CODE
    return Xdisagree, probability

In [65]:
Xdisagree, probability = compare()
assert type(Xdisagree) == list
print(Xdisagree)
assert probability >= 0.0 and probability <= 1.0

[('yes', 51, 'good')]


One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [12]:
class NearestMeanClassifier:
    def train(self, dataset):        
        """
        Trains the dataset and returns the machine learning model.
        
        Parameters
        -------------
        dataset: ((smoke: str, age: int, diet: str), label: str)
        
        Return
        -------------
        {
            "less": (smoke: int, age: int, diet: int),
            "more": (smoke: int, age: int, diet: int)
        }
        """
        
        # Here is the code

        """
        This is a helper method. It transforms the input `dataset` to a continuous data set.
        The map is defined as follows:
           - smoke == yes  ~> 1.0  
           - diet  == poor ~> 1.0
           
        Example
        -------------
        Input: (('yes', 34, 'poor'), label)
        Output: (1, 34, 1)
        """
        def transform(c):
            smoke, diet, age = 0, 0, c[0][1]
            if c[0][0] == "yes":
                smoke = 1
            if c[0][2] == "poor":
                diet = 1
            return (smoke, age, diet)
        
        """
        Example: [('yes', 34, 'poor'), ('yes', 34, 'good')] ~> [(1, 34, 1), (1, 34, 0) ]
        """
        healthyDataSet = list(map(lambda x: transform(x), list(filter (lambda x: x[1] == "less", dataset))))
        riskyDataSet = list(map(lambda x: transform(x), list(filter (lambda x: x[1] == "more", dataset))))
        
        """
        Takes a list of 3-dimensional vectors and returns the means vector.
        
        Parameters
        -------------
        dataset: (smoke: int, age: int, diet: int)
        
        Return
        -------------
        mean vector of the dataset: (average of smoke, average of age, average of diet)
        """
        def computeAverage(dataSet):
            avg = (0,0,0)
            size = float(len(dataSet))
            for n in dataSet: 
                avg = (avg[0] + n[0], avg[1] + n[1], avg[2] + n[2])
            return (avg[0] / size, avg[1]/ size, avg[2]/ size)
        
        """
        This is our trained machine learning model. It has two keys, each containing 
        the 3-dimensional features vector. 
        
        Example
        --------------
        {
                     SMOKER  AGE   DIET 
            "less": ( 0.33, 32.0, 0.222 ),
            "more": ( 0.57, 37.0,  0.57 ) 
        }
        """
        model = {
            "less": computeAverage(healthyDataSet), 
            "more": computeAverage(riskyDataSet)
        }
        
        return model

    def predict(self, x):
        # >>>>> YOUR CODE HERE
        """
        Parameter
        ------------
        x: (testVector: (smoke: str, age: int, diet: str), model: Dictionary)

        testVector ... a 3-dimensional vector (age) for which we want to predict the class label
        model      ... the model that is returned by the method `train()`
        """
        def transform(c):
            smoke, diet = 0, 0 
            if c[0] == "yes":
                smoke = 1
            if c[2] == "poor":
                diet = 1
            return (smoke, c[1], diet)
        
        # vector for which the class label is predicted
        testVector = transform(x[0])
        # the model that runs the classification
        model = x[1]
        
        """
        A plain distance function. Nothing special here.
        """
        def d(a,b):
            return (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2
        
        # Calculate the distances
        distanceToLess = d(testVector, model["less"]) # model["less"] contains the means vector of "less"
        distanceToMore = d(testVector, model["more"]) # model["more"] contains the means vector of "more"
        
        prediction = "less" if distanceToLess < distanceToMore else "more"
        # <<<<< END YOUR CODE
        return prediction

* Build an object of class `NearestMeanClassifier`, train it on the training data, and return the mean vector for each class. You should return a dictionary with keys `less` and `more`. Each key should correspond to a mean vector.

In [81]:
def build_and_train():
    # >>>>> YOUR CODE HERE
    W = NearestMeanClassifier()
    return W.train(gettrain())
    # <<<<< END YOUR CODE

* Predict the test data using the nearest mean classifier and return all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree. You should return a list containing tuples. Each tuple should contain the datapoint and the prediction i.e.

`[(('no', 50, 'good'), 'less'), ... ]`

In [80]:
def predict_test():
    # >>>>> YOUR CODE HERE
    
    prediction = []
    agreed_samples = []


    # list of 3-dimensional vectors (smoker: str, age: int, diet: str)
    testDataSet = gettest()
    #((smoke, age, diet), label)
    trainingDataSet = gettrain()
    
    """
    3-Step-Duc-Jacky-Laura machine learning algorithm
    ----------------------------------------------------
      (1) Initialize classifier
      (2) Train model
      (3) Predict
    """
    W = NearestMeanClassifier()
    model = W.train(trainingDataSet)
    prediction += ([(x, W.predict((x, model))) for x in testDataSet])
    
    # Decision Tree 
    prediction += ([(x, decision(x)) for x in testDataSet])
    
    # Nearest Neighbor
    prediction += ([(x, neighbor(x, trainingDataSet)) for x in testDataSet])
    
    """
    Assume the test data set contains 3 vectors. 
    prediction is a list and is structured as follows:
    
     0 1 2 3 4 5 6 7 8 
    [A|A|A|B|B|B|C|C|C]

    A... prediction of nearest mean classifier
    B... prediction of decision tree
    C... prediction of neigbor
    
    We compare the elements at the indeces [0], [0+len(testSet)], [0 + len(testSet) * 2] with each other.
    That means:
    
     0 1 2 3 4 5 6 7 8
    [A|A|A|B|B|B|C|C|C]
     X     X     X
     
     We do this for idx = 0,1,2, ..., len(testSet) - 1.
     
     0 1 2 3 4 5 6 7 8
    [A|A|A|B|B|B|C|C|C]
       x     X     X
    """
    
    n = len(testDataSet)
    for idx in range(0, n):
        if prediction[idx][1] == prediction[idx+n][1] == prediction[idx+2*n][1]:
            # if all predictions coincide add to agreed samples
            agreed_samples += prediction[idx]
            
    # <<<<< END YOUR CODE
    return agreed_samples

._.)/\(._.